In [1]:
def baselines():
    from ProgressPrinter import ProgressPrinter
    from FewShot import ZeroShotClassifier
    from PersonalizedCitation import train_loader
    import torch

    device = 'cpu'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=2)
    zero = ZeroShotClassifier()

    with ProgressPrinter('0 shot acc') as printer:
        for inputs, profiles, answers in train:
            with torch.no_grad():
                multichoices = [ ( i, train.choices ) for i in inputs ] 
                zeroguesses = zero(multichoices)
                zeroreward = torch.Tensor([ float(guess == answer) for guess, answer in zip(zeroguesses, answers) ]).mean().item()

            printer.addobs(zeroreward)

from Fork import SubProcess
with SubProcess() as process: process.parent or baselines()

n          0 shot acc      since     dt (s)
1                   1          1       1.29
2                   1          1       2.03
4                0.75        0.5       3.86
8               0.625        0.5       7.43
16              0.688       0.75       13.4
32              0.578      0.469       27.2
64              0.516      0.453       56.5
128             0.469      0.422        136
256             0.506      0.543        305
512             0.514      0.521        592
1024            0.505      0.496   1.19e+03
2048              0.5      0.495   2.32e+03
4096            0.495      0.489   4.64e+03
4841            0.493      0.483   5.46e+03
